In [1]:
import pandas as pd
from sodapy import Socrata
from uszipcode import SearchEngine # note this does an approximation of zipcode based upon a radius
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

# import pickle

In [2]:
pd.options.display.max_columns=20

In [3]:
client = Socrata("data.lacity.org", None)

results = client.get("7fvc-faax", limit=2000000)
# results = client.get("7fvc-faax", limit=200)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [4]:
results_df.head()

,:@computed_region_2dna_qi2s,:@computed_region_k96s_3jcv,:@computed_region_kqwf_mjcx,:@computed_region_qz3q_ghft,:@computed_region_tatf_ua23,:@computed_region_ur2y_g4cx,area_id,area_name,crm_cd,crm_cd_1,...,premis_desc,rpt_dist_no,status,status_desc,time_occ,vict_age,vict_descent,vict_sex,weapon_desc,weapon_used_cd
0,26,626,7,23667,1086,NaN,07,Wilshire,930,930,...,PARK/PLAYGROUND,0724,AA,Adult Arrest,1050,41,B,M,VERBAL THREAT,511
1,26,635,7,23667,332,NaN,07,Wilshire,626,626,...,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",0735,AA,Adult Arrest,0600,27,O,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400
2,26,633,6,23679,809,NaN,07,Wilshire,930,930,...,PARKING LOT,0721,AA,Adult Arrest,2350,20,H,F,VERBAL THREAT,511
3,52,612,12,23080,1229,NaN,07,Wilshire,626,626,...,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",0759,AA,Adult Arrest,2330,35,O,M,UNKNOWN WEAPON/OTHER WEAPON,500
4,69,651,12,23080,1105,NaN,07,Wilshire,236,236,...,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",0787,AO,Adult Other,2000,43,K,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400


In [5]:
# with open('results_df.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(results_df, f, pickle.HIGHEST_PROTOCOL)

In [6]:
# with open('results_df.pickle', 'rb') as f:
#     # The protocol version used is detected automatically, so we do not
#     # have to specify it.
#     results_df = pickle.load(f)

In [7]:
list(results_df)

[':@computed_region_2dna_qi2s',
 ':@computed_region_k96s_3jcv',
 ':@computed_region_kqwf_mjcx',
 ':@computed_region_qz3q_ghft',
 ':@computed_region_tatf_ua23',
 ':@computed_region_ur2y_g4cx',
 'area_id',
 'area_name',
 'crm_cd',
 'crm_cd_1',
 'crm_cd_2',
 'crm_cd_3',
 'crm_cd_4',
 'crm_cd_desc',
 'cross_street',
 'date_occ',
 'date_rptd',
 'dr_no',
 'location',
 'location_1',
 'mocodes',
 'premis_cd',
 'premis_desc',
 'rpt_dist_no',
 'status',
 'status_desc',
 'time_occ',
 'vict_age',
 'vict_descent',
 'vict_sex',
 'weapon_desc',
 'weapon_used_cd']

In [8]:
rows=len(results_df)
search = SearchEngine(simple_zipcode=True)
lon = []
lat = []
zipcode = []
year_single = '2000'
year = []
for i in range(rows):
    # single values for readability
    lattitude = float(results_df['location_1'][i]['coordinates'][1])
    longitude = float(results_df['location_1'][i]['coordinates'][0])
    year_single = results_df['date_occ'][i][:4]
    # append to arrays
    lon.append(longitude)
    lat.append(lattitude)
    try:
        zipcode.append(search.by_coordinates(lattitude, longitude, radius=5, returns=1)[0].zipcode)
    except:
        zipcode.append('00000')
        print(f'{i},{lattitude},{longitude}')
    year.append(year_single)
results_df['lat'] = lat
results_df['lon'] = lon
results_df['zipcode'] = zipcode
results_df['year'] = year

2513,0.0,0.0
12985,34.706,-118.0879
32768,0.0,0.0
40068,0.0,0.0
54647,0.0,0.0
142345,34.706,-118.0879
143901,0.0,0.0
143937,0.0,0.0
144892,0.0,0.0
144964,0.0,0.0
145194,0.0,0.0
145223,0.0,0.0
158157,0.0,0.0
181231,0.0,0.0
186642,34.576,-118.0115
194622,33.3427,-118.3258
202117,0.0,0.0
202226,0.0,0.0
202858,0.0,0.0
203448,0.0,0.0
203594,0.0,0.0
203820,0.0,0.0
203935,0.0,0.0
220141,0.0,0.0
241522,0.0,0.0
272819,0.0,0.0
274499,0.0,0.0
286349,0.0,0.0
321578,0.0,0.0
351718,34.6765,-118.1089
356908,0.0,0.0
361817,0.0,0.0
372580,0.0,0.0
405296,33.3427,-118.3258
405551,33.3427,-118.3258
421637,0.0,0.0
426250,0.0,0.0
426251,0.0,0.0
426252,0.0,0.0
426253,0.0,0.0
426254,0.0,0.0
426255,0.0,0.0
426256,0.0,0.0
426257,0.0,0.0
426258,0.0,0.0
426259,0.0,0.0
426260,0.0,0.0
426261,0.0,0.0
426262,0.0,0.0
426263,0.0,0.0
426264,0.0,0.0
426265,0.0,0.0
426266,0.0,0.0
426267,0.0,0.0
426268,0.0,0.0
426269,0.0,0.0
426270,0.0,0.0
434296,0.0,0.0
434312,0.0,0.0
434313,0.0,0.0
434314,0.0,0.0
434315,0.0,0.0
434316,0.

582470,0.0,0.0
582471,0.0,0.0
582472,0.0,0.0
582473,0.0,0.0
588366,0.0,0.0
588386,0.0,0.0
588387,0.0,0.0
588388,0.0,0.0
588389,0.0,0.0
588390,0.0,0.0
588391,0.0,0.0
588392,0.0,0.0
588393,0.0,0.0
588394,0.0,0.0
588395,0.0,0.0
588396,0.0,0.0
588397,0.0,0.0
588398,0.0,0.0
588399,0.0,0.0
588400,0.0,0.0
588401,0.0,0.0
588402,0.0,0.0
588403,0.0,0.0
588404,0.0,0.0
588405,0.0,0.0
588406,0.0,0.0
588407,0.0,0.0
588408,0.0,0.0
588409,0.0,0.0
588410,0.0,0.0
588411,0.0,0.0
588412,0.0,0.0
588413,0.0,0.0
588414,0.0,0.0
588415,0.0,0.0
588416,0.0,0.0
588417,0.0,0.0
588418,0.0,0.0
588419,0.0,0.0
592879,0.0,0.0
596751,0.0,0.0
598656,0.0,0.0
598657,0.0,0.0
598672,0.0,0.0
598673,0.0,0.0
598698,0.0,0.0
598729,0.0,0.0
598730,0.0,0.0
598731,0.0,0.0
598733,0.0,0.0
598734,0.0,0.0
598735,0.0,0.0
598736,0.0,0.0
598737,0.0,0.0
598738,0.0,0.0
598741,0.0,0.0
598742,0.0,0.0
598743,0.0,0.0
598744,0.0,0.0
598745,0.0,0.0
598746,0.0,0.0
598747,0.0,0.0
598748,0.0,0.0
598749,0.0,0.0
598750,0.0,0.0
598752,0.0,0.0
598753,0.0

In [9]:
len(results_df)

1931442

In [10]:
# remove lines with no corresponding zip code
results_df = results_df[results_df.zipcode != '00000']
len(results_df)

1930536

In [11]:
# drop irrelevant columns
results_df.drop(columns=[':@computed_region_2dna_qi2s',
 ':@computed_region_k96s_3jcv',
 ':@computed_region_kqwf_mjcx',
 ':@computed_region_qz3q_ghft',
 ':@computed_region_tatf_ua23',
 ':@computed_region_ur2y_g4cx',
 'crm_cd_1',
 'crm_cd_2',
 'crm_cd_3',
 'cross_street'], inplace=True)

In [12]:
results_df.head()

,area_id,area_name,crm_cd,crm_cd_4,crm_cd_desc,date_occ,date_rptd,dr_no,location,location_1,...,time_occ,vict_age,vict_descent,vict_sex,weapon_desc,weapon_used_cd,lat,lon,zipcode,year
0,07,Wilshire,930,NaN,CRIMINAL THREATS - NO WEAPON DISPLAYED,2010-03-12T00:00:00.000,2010-03-12T00:00:00.000,100700660,7600 BEVERLY BL,"{'type': 'Point', 'coordinates': [-118.3555, 3...",...,1050,41,B,M,VERBAL THREAT,511,34.0761,-118.3555,90036,2010
1,07,Wilshire,626,NaN,INTIMATE PARTNER - SIMPLE ASSAULT,2010-03-13T00:00:00.000,2010-03-13T00:00:00.000,100700661,600 S DETROIT ST,"{'type': 'Point', 'coordinates': [-118.3451, 3...",...,0600,27,O,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400,34.0649,-118.3451,90036,2010
2,07,Wilshire,930,NaN,CRIMINAL THREATS - NO WEAPON DISPLAYED,2010-03-23T00:00:00.000,2010-03-24T00:00:00.000,100700688,LA CIENEGA BL,"{'type': 'Point', 'coordinates': [-118.3765, 3...",...,2350,20,H,F,VERBAL THREAT,511,34.0730,-118.3765,90048,2010
3,07,Wilshire,626,NaN,INTIMATE PARTNER - SIMPLE ASSAULT,2010-03-23T00:00:00.000,2010-03-24T00:00:00.000,100700689,1100 NORTON AV,"{'type': 'Point', 'coordinates': [-118.3219, 3...",...,2330,35,O,M,UNKNOWN WEAPON/OTHER WEAPON,500,34.0516,-118.3219,90005,2010
4,07,Wilshire,236,NaN,INTIMATE PARTNER - AGGRAVATED ASSAULT,2010-04-05T00:00:00.000,2010-04-05T00:00:00.000,100700719,4400 W WASHINGTON BL,"{'type': 'Point', 'coordinates': [-118.3353, 3...",...,2000,43,K,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400,34.0399,-118.3353,90019,2010


In [13]:
file_name='los_angeles_crime_statistics.csv'
results_df.to_csv(file_name, encoding='utf-8')

In [ ]:
# dates

In [ ]:
# construct a basic timedelta and print it
print (timedelta(days=365, hours=0, minutes=0))
# print today's date
print ("today is: " + str(datetime.now()))
# print today's date one year from now
print ("one year from now it will be:" + str(datetime.now() - timedelta(days=365)))


In [ ]:
now = datetime.now()
print(now)
print(timedelta(days=365))
# 1year = datetime.now() - timedelta(days=365)
# 1year
print(type(now))

In [ ]:
df_0 = results_df

In [ ]:
# group by zipcode /crm_cd ; count number of crimes (dr_no) 
aggregation = { 
     'dr_no': 'nunique' 
}
df_1 = results_df.groupby(['zipcode','crm_cd']).agg(aggregation)
df_1

In [ ]:
df_1.plot(kind="line", figsize=(20,3))

In [ ]:
# group by year / zipcode ; count number of crimes (dr_no)
aggregation = { 
     'dr_no': 'nunique' 
}
df_2 = results_df.groupby(['year','zipcode']).agg(aggregation)
df_2

In [ ]:
fig = plt.figure()
ax = plt.axes()

x = np.linspace(0, 10, 1000)
ax.plot(x, np.sin(x));

In [ ]:
# group by year / zipcode ; count number of crimes (dr_no)
df_4 = results_df.sort_values(by=['zipcode'])
aggregation = { 
     'dr_no': 'count' 
}
df_3 = df_4.groupby(['zipcode']).agg(aggregation)
df_3 = df_3.sort_values(by=['dr_no'], ascending=False)
df_3 = df_3[:10]
df_3.reset_index(level=0, inplace=True)
df_3

In [ ]:
df_5 = df_3['zipcode']
df_5

In [ ]:
# rolling year

In [ ]:
# 5 years

In [ ]:
df_3 = df_3.sort_values(by='zipcode')
top_zipcodes = df_3.zipcode.unique()
top_zipcodes

In [ ]:
aggregation = { 
     'dr_no': 'nunique' 
}
zip_90012 = pd.DataFrame(df_0[df_0.zipcode == '90012'].groupby(['zipcode']).agg(aggregation))

# df_6.reset_index(level=0, inplace=True)
zip_90012

In [ ]:
plot('xlabel', 'ylabel', data=df_2)
plt.xlabel('Total Number of Rides (Per City)', fontdict=None, labelpad=None)
plt.ylabel('Average Fare ($)', fontdict=None, labelpad=None)
plt.title('Pyber Ride Sharing Data (2016)', fontdict=None, loc='center', pad=None)
plt.grid(b=None, which='major', axis='both')

In [ ]:
# income

In [ ]:
# home value

In [ ]:
result